In [1]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages (20.0.2)


In [2]:
!pip install pyspark

In [1]:
import pyspark

In [2]:
from pyspark import SparkContext 

In [3]:
sc = SparkContext.getOrCreate()


In [4]:
pairRDD = sc.parallelize([(1,2),(2,3),(2,4),(3,4),(3,5)])

In [15]:
reduceRDD = pairRDD.reduceByKey(lambda x,y : (x+y))
reduceRDD.collect()

[(1, 2), (2, 7), (3, 9)]

In [16]:
groupByRDD = pairRDD.groupByKey()
groupByRDD.collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x795de90>),
 (2, <pyspark.resultiterable.ResultIterable at 0x795d9f0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x795dfb0>)]

In [17]:
#Check partitions 
groupByRDD.getNumPartitions()

4

In [20]:
#Print out of groupByRDD
for t in groupByRDD.collect():
    print(t[0],[v for v in t[1]])

1 [2]
2 [3, 4]
3 [4, 5]


In [24]:
mapValuesRDD = pairRDD.mapValues(lambda x : x*x)
mapValuesRDD.collect()

[(1, 4), (2, 9), (2, 16), (3, 16), (3, 25)]

In [27]:
flatMapValuesRDD = pairRDD.flatMapValues(lambda x : range(x, 5))
flatMapValuesRDD.collect()

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (2, 4), (3, 4)]

In [30]:
keys = pairRDD.keys()
keys.collect()

[1, 2, 2, 3, 3]

In [31]:
values = pairRDD.values()
values.collect()
# check why two actions can not work in one cell of jupyter notebook

[2, 3, 4, 4, 5]

In [32]:
sortByKetRDD = pairRDD.sortByKey()
sortByKetRDD.collect()

[(1, 2), (2, 3), (2, 4), (3, 4), (3, 5)]

In [34]:
pairRDD2 = sc.parallelize([(1, 2),(3, 5)])

In [35]:
subtractByKeyRDD = pairRDD.subtractByKey(pairRDD2)
subtractByKeyRDD.collect()

[(2, 3), (2, 4)]

In [36]:
joinRDD = pairRDD.join(pairRDD2)
joinRDD.collect()

[(1, (2, 2)), (3, (4, 5)), (3, (5, 5))]

In [37]:
coGroupRDD = pairRDD.cogroup(pairRDD2)
coGroupRDD.collect()

[(1,
  (<pyspark.resultiterable.ResultIterable at 0x7fc44f0>,
   <pyspark.resultiterable.ResultIterable at 0x7fc46b0>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x7fc43b0>,
   <pyspark.resultiterable.ResultIterable at 0x7fc4e70>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x7fc4bf0>,
   <pyspark.resultiterable.ResultIterable at 0x7fc4970>))]

In [41]:
coGroupRDD.values().collect()

[(<pyspark.resultiterable.ResultIterable at 0x8075410>,
  <pyspark.resultiterable.ResultIterable at 0x8075a90>),
 (<pyspark.resultiterable.ResultIterable at 0x80753f0>,
  <pyspark.resultiterable.ResultIterable at 0x8075430>),
 (<pyspark.resultiterable.ResultIterable at 0x8075450>,
  <pyspark.resultiterable.ResultIterable at 0x8075470>)]

In [45]:
#Print out of coGroupRDD
for t in coGroupRDD.collect():
    for x in t[1]:
        print (y for y in x)
#print(t[0],[v for v in t[1]])

<generator object <genexpr> at 0x07DB4F70>
<generator object <genexpr> at 0x07DB4F70>
<generator object <genexpr> at 0x07DB4F70>
<generator object <genexpr> at 0x07DB4F70>
<generator object <genexpr> at 0x07DB4F70>
<generator object <genexpr> at 0x07DB4F70>


In [5]:
mapvValuesAvgRDD = pairRDD.mapValues(lambda x : (x,1))
mapvValuesAvgRDD.collect()

[(1, (2, 1)), (2, (3, 1)), (2, (4, 1)), (3, (4, 1)), (3, (5, 1))]

In [7]:
reduceByKeyAvgRDD =  mapvValuesAvgRDD.reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
reduceByKeyAvgRDD.collect()

[(1, (2, 1)), (2, (7, 2)), (3, (9, 2))]

In [8]:
lines = sc.textFile("README_test.md")

In [14]:
words = lines.flatMap(lambda x : x.split(" "))
result = words.map(lambda x: (x,1)).reduceByKey(lambda x,y : (x+y))
result.take(10)

[('#', 1),
 ('Apache', 1),
 ('Spark', 15),
 ('', 72),
 ('is', 7),
 ('It', 2),
 ('provides', 1),
 ('high-level', 1),
 ('APIs', 1),
 ('in', 5)]

In [31]:
# countByValue is action 
wordCount = lines.flatMap(lambda x : x.split(" ")).countByValue()
#wordCount

In [42]:
#combineByKey
#  (lambda x : (x,1)) will create like (2,(2,1))
#  (lambda x, y : (x[0] + y, x[1]+1)) will be like for key 2 is (2,1),3 : (2+3), (1+1)
#  (lambda x,y : (x[0]+y[0], x[1]+y[1])) will be like (2,1),(3,1): ((2+3),(1+1))
sumCount = pairRDD.combineByKey((lambda x : (x,1)),
                                (lambda x, y : (x[0] + y, x[1]+1)),
                                (lambda x,y : (x[0]+y[0], x[1]+y[1]))
                               )

In [43]:
sumCount.take(10)

[(1, (2, 1)), (2, (7, 2)), (3, (9, 2))]

In [49]:
#sumCount.values
resultSumCountRDD = sumCount.map(lambda(key, (x,y)) : (key , x / y))
#resultSumCountRDD.collect()

SyntaxError: invalid syntax (<ipython-input-49-e265078603d0>, line 2)

In [46]:
resultSumCountRDD.values()

PythonRDD[68] at RDD at PythonRDD.scala:53

In [5]:
partitionNumber = pairRDD.getNumPartitions()
partitionNumber

4

In [9]:
#newPartitionNumber = pairRDD.repartition(2)
newPartitionNumber = pairRDD.coalesce(3)
newPartitionNumber.getNumPartitions()

3

In [ ]:
# Patitioning reduce data shuffle over netowrk.
# use cache()/persist() on partioned RDD so that same RDD did not get evaluated again again and it saves in memory 
# if parent RDD is partitioned and we need to maintain partioned then use mapValues() instead of map() and use
# flatMapvalues() intead of flatMap() 